In [1]:
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import os
import requests
from os import environ

requests.packages.urllib3.disable_warnings()

plotly.tools.set_credentials_file(username='pusiol', api_key='m6iurl7f89')

/media/Gui2/thermix/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/media/Gui2/thermix/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [2]:
default_filepath = '/media/Gui2/thermix/ARTraining/trained_models/thermix_13_06-06-2016/output_thermix_13.file'
filepath = environ.get("output_filepath",default_filepath)

In [3]:
with open(filepath,'r') as f:
    # get all lines in file
    lines = f.readlines()

    # remove all those weird chars
    newlines = []
    for line in lines:
        newline = line.replace('\x08',' ')
        newlines.append(filter(None, newline.split('  ')))

    training_batch_count = None
    testing_batch_count = None
    for line in newlines:
        if training_batch_count and testing_batch_count:
            break;
            
        for s in line:
            if training_batch_count and testing_batch_count:
                break;

            if not training_batch_count and "stochastic gradient descent with miss rate" in s:
                # txt = "% at 01 / 110 ("
                txt = s[s.index("% at "):]
                txt = txt[:txt.index("(")]
                
                # training_batch_count = 110
                training_batch_count = int(txt.split("/")[1][1:-1])
            
            if not testing_batch_count and "=> with miss rate" in s:
                # txt = "at 01 / 24 ("
                txt = s[s.index("% at "):]
                txt = txt[:txt.index("(")]
                
                # testing_batch_count = 110
                testing_batch_count = int(txt.split("/")[1][1:-1])
            
    if not training_batch_count or not testing_batch_count:
        raise "malformed file"
    #now search for 75 / 75 (training) y 16 / 16 (validation)
    training_error_text = '%d / %d' % (training_batch_count, training_batch_count)
    validation_error_text = '%d / %d' % (testing_batch_count, testing_batch_count)

    print training_error_text
    print validation_error_text
    
    training_error_lines = []
    validation_error_lines = []
    for a in newlines:
        for s in a:
            if training_error_text in s:
                training_error_lines.append(s)
            if validation_error_text in s:
                validation_error_lines.append(s)

    # get matrix for validation loss
    v_loss = []
    for v_data in validation_error_lines:
        index_rate = v_data.index('rate')
        v_loss.append(float(v_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))

    # get matrix for training loss
    t_loss = []
    for t_data in training_error_lines:
        index_rate = t_data.index('rate')
        t_loss.append(float(t_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))


110 / 110
17 / 17


In [5]:
#plot data
data = []
Ys = [('validation sparse softmax cross entropy loss', v_loss), ('training sparse softmax cross entropy loss', t_loss)]
for title, _Y in Ys:
    # t = 1 -> validation data
    _X = range(1,len(_Y)+1) # _X = epoch number
    mom = Scatter(x=_X,y=_Y, mode ='lines+markers', name =title ,line = dict(width = 4))
    data.append(mom)

layout = Layout(title='%s loss function'%os.path.basename(filepath).split(".")[0], xaxis = dict(title = 'Epoch'), yaxis = dict(title = 'miss rate'), )

# Plot and embed in ipython notebook!
fig = dict(data=data, layout=layout)

#upload to guido's account
py.iplot(fig, filename="lastest_trained_model_info")
